In [ ]:
import pandas as pd
import numpy as np

CONFIG = {
    "widetable_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_path": r'./reverse_strategy_backtest.csv',
    # 反向选股条件
    "auction_ratio_threshold": 0.8,  # 竞价量比≤0.8（资金冷淡）
    "industry_cold_rank": 0.5,       # 行业热度后50%
    "3d_loss_threshold": -10.0,      # 前3日跌幅≥10%（超跌）
    # 反向交易规则
    "buy_down_ratio": 0.02,          # 跌到前5日低点2%内买入
    "stop_profit": 3.0,              # 反弹≥3%止盈
    "stop_loss": -2.0,               # 再跌≤-2%止损
    "initial_fund": 100000.0
}

def load_data():
    df = pd.read_parquet(CONFIG["widetable_path"])
    df["date"] = pd.to_datetime(df["date"])
    df = df[df["paused"] == 0].copy()
    # 计算反向所需因子
    df["auction_vol_5d_mean"] = df.groupby("stock_code")["auc_volume"].transform(
        lambda x: x.rolling(5, min_periods=3).mean().shift(1).replace(0, 0.0001)
    )
    df["auc_volume_ratio"] = df["auc_volume"] / df["auction_vol_5d_mean"]
    df["daily_return"] = (df["close"] / df["pre_close"] - 1) * 100
    df["3d_return"] = df.groupby("stock_code")["daily_return"].transform(lambda x: x.rolling(3).sum().shift(1))
    df["ma5"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(5).mean())
    df["ma20"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(20).mean())
    df["5d_low"] = df.groupby("stock_code")["low"].transform(lambda x: x.rolling(5).min().shift(1))  # 前5日低点
    # 行业热度排名（取后50%）
    industry_rank = df.groupby(["date", "sw_l1_industry_name"])["daily_return"].mean().reset_index()
    industry_rank["industry_percentile"] = industry_rank.groupby("date")["daily_return"].rank(pct=True)
    df = df.merge(industry_rank[["date", "sw_l1_industry_name", "industry_percentile"]], on=["date", "sw_l1_industry_name"], how="left")
    return df

def select_reverse_stocks(df):
    all_selections = []
    valid_dates = sorted(df["date"].unique())
    for date in valid_dates:
        daily_df = df[df["date"] == date].copy()
        # 反向选股：超跌+资金冷淡+非热点
        daily_df = daily_df[
            (daily_df["auc_volume_ratio"] <= CONFIG["auction_ratio_threshold"]) &
            (daily_df["ma5"] < daily_df["ma20"]) &  # 均线空头
            (daily_df["3d_return"] <= CONFIG["3d_loss_threshold"]) &  # 超跌
            (daily_df["industry_percentile"] <= CONFIG["industry_cold_rank"]) &  # 非热点行业
            (daily_df["daily_return"].shift(1) >= -1.0)  # 前1日跌幅收窄
        ].copy()
        if len(daily_df) == 0:
            continue
        # 每日选前5只超跌股
        daily_df["oversold_score"] = daily_df["3d_return"].rank(ascending=True)  # 跌幅越大得分越高
        selected = daily_df.nsmallest(5, "oversold_score")
        selected["selection_date"] = date
        all_selections.append(selected[["selection_date", "stock_code", "5d_low"]])
    return pd.concat(all_selections, ignore_index=True)

def backtest_reverse(selections, df):
    price_map = df.set_index(["date", "stock_code"])[["open", "close", "low"]].to_dict("index")
    fund = CONFIG["initial_fund"]
    records = []
    for _, row in selections.iterrows():
        stock = row["stock_code"]
        t_date = row["selection_date"]
        t1_date = t_date + pd.Timedelta(days=1)
        t2_date = t1_date + pd.Timedelta(days=1)
        # 获取T+1数据
        t1_data = price_map.get((t1_date, stock))
        if not t1_data:
            continue
        t1_low = t1_data["low"]
        t1_open = t1_data["open"]
        target_buy_price = row["5d_low"] * (1 + CONFIG["buy_down_ratio"])  # 目标买入价（前5日低点+2%）
        # T+1日跌到目标价附近买入
        if t1_low <= target_buy_price <= t1_open:
            buy_price = target_buy_price
        elif t1_low <= target_buy_price:
            buy_price = t1_low
        else:
            continue  # 没跌到目标价，放弃买入
        # 计算仓位
        shares = int((fund * 0.2) // (buy_price * 100)) * 100  # 单票20%仓位
        # T+2日卖出
        t2_data = price_map.get((t2_date, stock), {})
        sell_price = t2_data.get("close", t1_data["close"])
        return_rate = (sell_price / buy_price - 1) * 100
        # 止盈止损
        if return_rate >= CONFIG["stop_profit"]:
            sell_price = buy_price * (1 + CONFIG["stop_profit"]/100)
            return_rate = CONFIG["stop_profit"]
        elif return_rate <= CONFIG["stop_loss"]:
            sell_price = buy_price * (1 + CONFIG["stop_loss"]/100)
            return_rate = CONFIG["stop_loss"]
        # 计算收益
        profit = (sell_price - buy_price) * shares
        fund += profit
        records.append({"return_rate": return_rate, "fund_after": fund})
    # 输出结果
    backtest_df = pd.DataFrame(records)
    valid_returns = backtest_df["return_rate"].dropna()
    win_rate = len(valid_returns[valid_returns>0])/len(valid_returns)*100 if len(valid_returns) else 0
    profit_loss_ratio = abs(valid_returns[valid_returns>0].mean()/valid_returns[valid_returns<=0].mean()) if len(valid_returns[valid_returns<=0]) else 0
    print(f"初始资金：{CONFIG['initial_fund']:.2f} → 最终资金：{fund:.2f}")
    print(f"胜率：{win_rate:.2f}% | 盈亏比：{profit_loss_ratio:.2f}")
    return backtest_df

# 执行反向策略
if __name__ == "__main__":
    df = load_data()
    selections = select_reverse_stocks(df)
    backtest_result = backtest_reverse(selections, df)

数据加载完成：678个交易日，5284只股票
选股完成：4053条记录，平均每日6.0只

🎯 高胜率高盈亏比策略回测结果
初始资金：100000.00元 → 最终资金：32111.00元
累计收益率：-67.89%
总交易笔数：3230 | 盈利笔数：1321 | 亏损笔数：1909
胜率：40.90% ❌（目标＞50%）
盈亏比：1.22 ❌（目标＞1.9）
平均盈利：4.71% | 平均亏损：-3.85%
